In [1]:
from utilities.dataloader import VOC
from imgaug import augmenters as iaa

import torchvision.transforms as transform
from utilities.augmentation import Augmenter

from torch.utils.data import dataloader
import torch
import torch.nn as nn

import yolov1
from yolov1 import detection_loss_4_yolo

from utilities.dataloader import detection_collate
from utilities.utils import save_checkpoint

#print(torch.cuda.is_available())

import time

import os



#data_path = "/home/ali/VOCdevkit/VOC2012"
#class_path = "/home/ali/VOCdevkit/VOC2012/voc.names"
data_path = "/home/ali/VOCdevkit/VOC2012_1500"
class_path = "/home/ali/VOCdevkit/VOC2012_1500/voc.names"

batch_size = 10
#device = 'cpu'
device = 'cuda'
check_point_path = "/home/ali/VOCdevkit/VOC2012_1500/save_model/"

use_visdom=True

seq = iaa.Sequential([])
compsed = transform.Compose([Augmenter(seq)])

with open(class_path) as f:
    class_list = f.read().splitlines()

train_data = VOC(root=data_path, transform=compsed, class_path=class_path)

train_loader = dataloader.DataLoader(dataset=train_data,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     collate_fn=detection_collate)

# model


dropout = 0.4
num_class = 20
learning_rate = .4
num_epochs = 100

net = yolov1.YOLOv1(params={"dropout": dropout,
                            "num_class": num_class})


if device == 'cpu':
    model = nn.DataParallel(net).cpu()
else:
    model=torch.nn.DataParallel(net).cuda()



optimaizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                              weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimaizer, gamma=0.95)

total_step = len(train_loader)

# print(total_step)

total_train_step = num_epochs * total_step

for epoch in range(1, num_epochs + 1):

    if (epoch == 200) or (epoch == 400):
        scheduler.step()
        
    start = time.time()
    for i, (images, labels, sizes) in enumerate(train_loader):
        current_train_step = (epoch - 1) * total_step + i + 1
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # calc Loss
        loss, \
        obj_coord1_loss, \
        obj_size1_loss, \
        obj_class_loss, \
        noobjness1_loss, \
        objness1_loss = detection_loss_4_yolo(outputs, labels, device)
        # print(current_train_step)

        # backward and optimze
        optimaizer.zero_grad()
        loss.backward()
        optimaizer.step()
        
        end = time.time()
        print(
            'epoch: [{}/{}], total step:[{}/{}] , batchstep [{}/{}], lr: {},'
            'total_loss: {:.4f}, objness1: {:.4f}, class_loss: {:.4f} Time: {:0.3f} seconds'.format(epoch, num_epochs,
                                                                              current_train_step, total_train_step,
                                                                              i + 1, total_step, learning_rate,
                                                                              loss.item(), obj_coord1_loss,
                                                                              obj_size1_loss, obj_class_loss, end-start))
        start = time.time()
     
                                                                        


    if (epoch %  20 == 0):
        '''
        torch.save({'test': epoch}, 'cc.zip')
        print("Saved...")

        '''
        save_checkpoint({'epoch':epoch+1,
                             'arch': "YOLOv1",
                             'state_dict':model.state_dict(),
                             },False, filename=os.path.join(check_point_path,'ep{:05d}_loss{:.04f}_lr{}.pth.tar'.format(epoch,
                                                            loss.item(),learning_rate,))
                            )
        print("The check point is saved")




Image Folder : /home/ali/VOCdevkit/VOC2012_1500/JPEGImages
Label Folder : /home/ali/VOCdevkit/VOC2012_1500/Annotations



VOC Parsing:   |████████████████████████████████████████| 100.0% (1500/1500)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (1500/1500)  Complete



/home/ali/Ali_Files/Yolo_July/YOLO_V1_B/utilities/augmentation.py:25: DeprecationWarning: Method `BoundingBoxesOnImage.cut_out_of_image()` is deprecated. Use `BoundingBoxesOnImage.clip_out_of_image()` instead. clip_out_of_image() has the exactly same interface.
  bbs_aug = bbs_aug.remove_out_of_image().cut_out_of_image()


epoch: [1/100], total step:[1/15000] , batchstep [1/150], lr: 0.4,total_loss: 38.9683, objness1: 8.7810, class_loss: 12.1434 Time: 11.796 seconds
epoch: [1/100], total step:[2/15000] , batchstep [2/150], lr: 0.4,total_loss: 15043532947456.0000, objness1: 7355100561408.0000, class_loss: 7688432910336.0000 Time: 20.300 seconds
epoch: [1/100], total step:[3/15000] , batchstep [3/150], lr: 0.4,total_loss: 4640133349376.0000, objness1: 2314201464832.0000, class_loss: 2325932146688.0000 Time: 29.700 seconds
epoch: [1/100], total step:[4/15000] , batchstep [4/150], lr: 0.4,total_loss: 1884447571968.0000, objness1: 980411744256.0000, class_loss: 904035893248.0000 Time: 30.700 seconds
epoch: [1/100], total step:[5/15000] , batchstep [5/150], lr: 0.4,total_loss: 1437459546112.0000, objness1: 712107098112.0000, class_loss: 725352382464.0000 Time: 16.300 seconds
epoch: [1/100], total step:[6/15000] , batchstep [6/150], lr: 0.4,total_loss: 547620487168.0000, objness1: 315811921920.0000, class_loss:

epoch: [1/100], total step:[48/15000] , batchstep [48/150], lr: 0.4,total_loss: 31003863040.0000, objness1: 15294450688.0000, class_loss: 15709413376.0000 Time: 46.691 seconds
epoch: [1/100], total step:[49/15000] , batchstep [49/150], lr: 0.4,total_loss: 7485469696.0000, objness1: 4073241856.0000, class_loss: 3412227840.0000 Time: 31.971 seconds
epoch: [1/100], total step:[50/15000] , batchstep [50/150], lr: 0.4,total_loss: 11249444864.0000, objness1: 5857175552.0000, class_loss: 5392268800.0000 Time: 25.918 seconds
epoch: [1/100], total step:[51/15000] , batchstep [51/150], lr: 0.4,total_loss: 7863899648.0000, objness1: 3781862144.0000, class_loss: 4082037504.0000 Time: 22.859 seconds
epoch: [1/100], total step:[52/15000] , batchstep [52/150], lr: 0.4,total_loss: 25098022912.0000, objness1: 12162571264.0000, class_loss: 12935451648.0000 Time: 30.963 seconds
epoch: [1/100], total step:[53/15000] , batchstep [53/150], lr: 0.4,total_loss: 12955091968.0000, objness1: 6276383744.0000, cla

...

epoch: [99/100], total step:[14813/15000] , batchstep [113/150], lr: 0.4,total_loss: 17082398801920.0000, objness1: 8111824306176.0000, class_loss: 8970575020032.0000 Time: 3.700 seconds
epoch: [99/100], total step:[14814/15000] , batchstep [114/150], lr: 0.4,total_loss: 9463541006336.0000, objness1: 4303783460864.0000, class_loss: 5159757283328.0000 Time: 4.600 seconds
epoch: [99/100], total step:[14815/15000] , batchstep [115/150], lr: 0.4,total_loss: 13100697780224.0000, objness1: 7380088651776.0000, class_loss: 5720609128448.0000 Time: 6.000 seconds
epoch: [99/100], total step:[14816/15000] , batchstep [116/150], lr: 0.4,total_loss: 8012156108800.0000, objness1: 3743390892032.0000, class_loss: 4268764692480.0000 Time: 3.600 seconds
epoch: [99/100], total step:[14817/15000] , batchstep [117/150], lr: 0.4,total_loss: 9403679899648.0000, objness1: 4776942632960.0000, class_loss: 4626737790976.0000 Time: 5.800 seconds
epoch: [99/100], total step:[14818/15000] , batchstep [118/150], lr:

epoch: [100/100], total step:[14857/15000] , batchstep [7/150], lr: 0.4,total_loss: 24687050489856.0000, objness1: 12119213867008.0000, class_loss: 12567837671424.0000 Time: 7.600 seconds
epoch: [100/100], total step:[14858/15000] , batchstep [8/150], lr: 0.4,total_loss: 42813966254080.0000, objness1: 22943847415808.0000, class_loss: 19870120935424.0000 Time: 6.100 seconds
epoch: [100/100], total step:[14859/15000] , batchstep [9/150], lr: 0.4,total_loss: 21449769222144.0000, objness1: 10462304403456.0000, class_loss: 10987465867264.0000 Time: 7.400 seconds
epoch: [100/100], total step:[14860/15000] , batchstep [10/150], lr: 0.4,total_loss: 28366637891584.0000, objness1: 15111587627008.0000, class_loss: 13255050264576.0000 Time: 10.100 seconds
epoch: [100/100], total step:[14861/15000] , batchstep [11/150], lr: 0.4,total_loss: 23148032425984.0000, objness1: 11659573723136.0000, class_loss: 11488457654272.0000 Time: 6.783 seconds
epoch: [100/100], total step:[14862/15000] , batchstep [1

epoch: [100/100], total step:[14901/15000] , batchstep [51/150], lr: 0.4,total_loss: 37736891285504.0000, objness1: 19134872027136.0000, class_loss: 18602021355520.0000 Time: 7.400 seconds
epoch: [100/100], total step:[14902/15000] , batchstep [52/150], lr: 0.4,total_loss: 10066941968384.0000, objness1: 5121614282752.0000, class_loss: 4945327161344.0000 Time: 4.900 seconds
epoch: [100/100], total step:[14903/15000] , batchstep [53/150], lr: 0.4,total_loss: 16463811313664.0000, objness1: 7431426932736.0000, class_loss: 9032384380928.0000 Time: 4.600 seconds
epoch: [100/100], total step:[14904/15000] , batchstep [54/150], lr: 0.4,total_loss: 8298828398592.0000, objness1: 4385158201344.0000, class_loss: 3913669935104.0000 Time: 3.900 seconds
epoch: [100/100], total step:[14905/15000] , batchstep [55/150], lr: 0.4,total_loss: 29891764420608.0000, objness1: 12719301328896.0000, class_loss: 17172464140288.0000 Time: 7.400 seconds
epoch: [100/100], total step:[14906/15000] , batchstep [56/150

epoch: [100/100], total step:[14945/15000] , batchstep [95/150], lr: 0.4,total_loss: 10384106848256.0000, objness1: 4479587975168.0000, class_loss: 5904518348800.0000 Time: 5.800 seconds
epoch: [100/100], total step:[14946/15000] , batchstep [96/150], lr: 0.4,total_loss: 16803247947776.0000, objness1: 8901900632064.0000, class_loss: 7901347315712.0000 Time: 4.588 seconds
epoch: [100/100], total step:[14947/15000] , batchstep [97/150], lr: 0.4,total_loss: 24167942455296.0000, objness1: 12479633555456.0000, class_loss: 11688309948416.0000 Time: 5.700 seconds
epoch: [100/100], total step:[14948/15000] , batchstep [98/150], lr: 0.4,total_loss: 40066848129024.0000, objness1: 20683337760768.0000, class_loss: 19383510368256.0000 Time: 5.500 seconds
epoch: [100/100], total step:[14949/15000] , batchstep [99/150], lr: 0.4,total_loss: 24425581772800.0000, objness1: 13748182974464.0000, class_loss: 10677398798336.0000 Time: 6.900 seconds
epoch: [100/100], total step:[14950/15000] , batchstep [100

epoch: [100/100], total step:[14989/15000] , batchstep [139/150], lr: 0.4,total_loss: 36402775457792.0000, objness1: 17265155112960.0000, class_loss: 19137621393408.0000 Time: 8.800 seconds
epoch: [100/100], total step:[14990/15000] , batchstep [140/150], lr: 0.4,total_loss: 12846663467008.0000, objness1: 6863613591552.0000, class_loss: 5983049875456.0000 Time: 5.800 seconds
epoch: [100/100], total step:[14991/15000] , batchstep [141/150], lr: 0.4,total_loss: 15562195337216.0000, objness1: 6615251550208.0000, class_loss: 8946944311296.0000 Time: 5.800 seconds
epoch: [100/100], total step:[14992/15000] , batchstep [142/150], lr: 0.4,total_loss: 20851109920768.0000, objness1: 10371668639744.0000, class_loss: 10479442329600.0000 Time: 8.300 seconds
epoch: [100/100], total step:[14993/15000] , batchstep [143/150], lr: 0.4,total_loss: 25953155678208.0000, objness1: 13455706816512.0000, class_loss: 12497449910272.0000 Time: 8.100 seconds
epoch: [100/100], total step:[14994/15000] , batchstep